## prepare masks

In [ ]:
! unzip "/content/drive/MyDrive/Updated_BraTSData_SSA_only.zip"

Archive:  /content/drive/MyDrive/Updated_BraTSData_SSA_only.zip
replace ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
! unzip "/content/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData.zip"

Archive:  /content/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData.zip
replace ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2/BraTS-SSA-00121-000/BraTS-SSA-00121-000-seg.nii.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
print("masks no_image")

import os
import torch
import numpy as np

from tqdm import tqdm
import nibabel as nib
from skimage import transform



# from SurfaceDice import compute_dice_coefficient

# set seeds
torch.manual_seed(2023)
np.random.seed(2023)


# create_folders
base = "/content/"
os.makedirs(base + "BraTS_data_africa", exist_ok=True)


os.makedirs(base + "BraTS_data_africa/mask/z", exist_ok=True)
os.makedirs(base + "BraTS_data_africa/mask/x", exist_ok=True)
os.makedirs(base + "BraTS_data_africa/mask/y", exist_ok=True)

src_folder = "/content/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2/"
image_size = 256

files = os.listdir(src_folder)
for i in tqdm(files):
  img =  nib.load(src_folder + i + "/"+ i +"-seg.nii.gz")
  img = img.get_fdata()
  num_slices_z = img.shape[2]
  num_slices_x = img.shape[0]
  num_slices_y = img.shape[1]



  for s in range(num_slices_z):
    image = img[:,:,s]
    np.save(base + "BraTS_data_africa/mask/z/"+i+"_"+ str(s), image)

  # for s in range(num_slices_x):
  #   image = img[s,:,:]
  #   np.save(base + "BraTS_data_africa/mask/x/"+i+"_"+ str(s), image)

  # for s in range(num_slices_y):
  #   image = img[:,s,:]
  #   np.save(base + "BraTS_data_africa/mask/y/"+i+"_"+ str(s), image)


masks no_image


100%|██████████| 60/60 [00:19<00:00,  3.02it/s]


In [ ]:
! pip install monai

In [ ]:
import sys
!{sys.executable} -m pip install opencv-python matplotlib
!{sys.executable} -m pip install 'git+https://github.com/Noha-Magdy/segment-anything.git'

!mkdir images
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg
!wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/groceries.jpg

!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth


  Cloning https://github.com/Noha-Magdy/segment-anything.git to /tmp/pip-req-build-6dzxj8_z
  Running command git clone --filter=blob:none --quiet https://github.com/Noha-Magdy/segment-anything.git /tmp/pip-req-build-6dzxj8_z
  Resolved https://github.com/Noha-Magdy/segment-anything.git to commit 98439a88601f4290b63152ab48b1efc88e4c6cd9
  Preparing metadata (setup.py) ... done
mkdir: cannot create directory ‘images’: File exists
--2023-08-07 16:24:32--  https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/truck.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 271475 (265K) [image/jpeg]
Saving to: ‘images/truck.jpg.1’

truck.jpg.1         100%[===================>] 265.11K  --.-KB/s    in 0.004s  

2023-08-07 16:24:32 (

In [ ]:
# saved 2d data
print("2d  multiclass")
import os, argparse
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import Compose, ToTensor, Resize
from torch.utils.data import Subset

from tqdm import tqdm
# from torchsummary import summary
import monai
from segment_anything import SamPredictor, sam_model_registry
from segment_anything.utils.transforms import ResizeLongestSide
# import wandb
import nibabel as nib
from segment_anything.utils.transforms import ResizeLongestSide
import torch.nn.functional as F
from skimage import transform, io, segmentation
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import random, json
from collections import defaultdict

torch.manual_seed(2023)
np.random.seed(2023)

2d  multiclass


In [ ]:
def get_bounding_box(mask):

	y_indices, x_indices = np.where(mask > 0)
	if len(y_indices) == 0:
		return [0,0,0,0]
	x_min, x_max = np.min(x_indices), np.max(x_indices)
	y_min, y_max = np.min(y_indices), np.max(y_indices)
	# add perturbation to bounding box coordinates
	H, W = mask.shape
	x_min = max(0, x_min - np.random.randint(0, 20))
	x_max = min(W, x_max + np.random.randint(0, 20))
	y_min = max(0, y_min - np.random.randint(0, 20))
	y_max = min(H, y_max + np.random.randint(0, 20))
	# bboxes = np.array([x_min, y_min, x_max, y_max])

	return [x_min, y_min, x_max, y_max]




model_type = 'vit_h'
checkpoint = 'sam_vit_h_4b8939.pth'
# sam_model = sam_model_registry[model_type](checkpoint=checkpoint)#.to('cuda')



embedding_path = "/content/drive/MyDrive/miccai_data/BraTS_data_africa/"
ground_truth_path = "/content/BraTS_data_africa/mask/"


In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=hXEbSnB8gqbB9mpIYEf8sqLciiLFf3&prompt=consent&access_type=offline&code_challenge=6NFiYQqumlS-fVNy4jfLU_40p36gkKzfcBiIjoUP-iQ&code_challenge_method=S256

Enter authorization code: 

Command killed by keyboard interrupt

^C


In [ ]:
!sudo zypper install gcc python3-devel
!sudo pip uninstall crcmod
!sudo pip install --no-cache-dir -U crcmod

sudo: zypper: command not found
Found existing installation: crcmod 1.7
Uninstalling crcmod-1.7:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/crcmod-1.7.dist-info/*
    /usr/local/lib/python3.10/dist-packages/crcmod/*
Proceed (Y/n)? ERROR: Operation cancelled by user


In [ ]:
!mkdir BraTS_data_africa

mkdir: cannot create directory ‘BraTS_data_africa’: File exists


In [ ]:
!gcloud config set project 	crafty-nova-394910
!gsutil -m cp -r gs://embedding_spark/BraTS_data_africa/t1/t1 ./BraTS_data_africa/
!gsutil -m cp -r gs://embedding_spark/BraTS_data_africa/t2 ./BraTS_data_africa/
!gsutil -m cp -r gs://embedding_spark/BraTS_data_africa/flair ./BraTS_data_africa/

In [ ]:

all_cases = {}
case_idxs = defaultdict(lambda:[])
idx = 0
for d in ["z"]:
		for m in ["flair", "t1", "t2"]:
				temp_cases = os.listdir(f"./BraTS_data_africa/{m}/{d}/embeddings")
				for c in temp_cases:
					name = c.split("_")[0]
					case_idxs[name].append(idx)
					all_cases[idx] = embedding_path + m + "/" + d + "/embeddings/" + c
				#   print(idx)
					idx+=1


# all_cases_names = case_idxs.keys()
all_cases_names = list(case_idxs.keys())
print("all_cases_idxs", len(all_cases_names))

all_cases_idxs 43


In [ ]:
def load_split(all_cases, selected_cases, case_idxs):
		res = {}
		final_idx=0
		for case_name in selected_cases:
				idxs = case_idxs[case_name]
				for idx in idxs:
						res[final_idx] = all_cases[idx]
						final_idx += 1
		return res

def filter_split_black_masks(train_list):
  ground_truth_path = "./BraTS_data_africa/mask/"
  with open('/content/data.json', 'r') as fp:
    data = json.load(fp)

  res = {}
  for idx in train_list:
    modality = train_list[idx].split("/")[7]
    # print(modality)
    name = train_list[idx].split("/")[9]
    if name == ".ipynb_checkpoints": continue
    # print(name)
    name = name.split(".")[0]
    name, slice_num = name.split("_")

		# print("[name][modality][int(idx)]", [name],[modality],[int(slice_num)])

    if data[name][modality][slice_num]:
      res[idx] = train_list[idx]

  final_res = {}
  idx=0
  for x in res:
    final_res[idx]=res[x]
    idx+=1
  return final_res

In [ ]:

# train_cases = random.sample(all_cases_names, int(0.7*len(all_cases_names)))
train_cases = all_cases_names[:int(0.7*len(all_cases_names))]
train_list = load_split(all_cases, train_cases, case_idxs)#{idx:all_cases[key] for idx, key in enumerate(train_cases)}

train_cases_dict = {name:1 for name in train_cases}
val_cases = [i for i in all_cases_names if i not in train_cases_dict]
val_list = load_split(all_cases, val_cases, case_idxs)

train_list = filter_split_black_masks(train_list)
val_list = filter_split_black_masks(val_list)

print("len(train_cases)", len(train_cases))
print("len(train_list.keys())", len(train_list.keys()))

print("len(val_cases)", len(val_cases))
print("len(val_list.keys())", len(val_list.keys()))

len(train_cases) 30
len(train_list.keys()) 6756
len(val_cases) 13
len(val_list.keys()) 2919


In [ ]:
# train_list[30]

In [ ]:
def preprocess(x: torch.Tensor) -> torch.Tensor:
	"""Normalize pixel values and pad to a square input."""
	# Normalize colors

	# Pad
	h, w = x.shape[-2:]
	padh = 256 - h
	padw = 256 - w
	x = F.pad(x, (0, padw, 0, padh))
	return x

class BraTSDataset(Dataset):

		def __init__(self, ground_truth_path, embedding_path, list_of_data, sam_model, device='cuda'):
				"""
				Args:
						embedding_path (string): Path to  images.
						masks_path (string): path to masks.
						transform (callable, optional): Optional transform to be applied
								on a sample.
				"""
				self.ground_truth_path = ground_truth_path
				self.device = device
				self.list_of_data = list_of_data
				self.embedding_path = embedding_path
				self.sam_model = sam_model

		def __len__(self):
				return len(self.list_of_data)

		def __getitem__(self, idx):


			embedding = np.load(self.list_of_data[idx])
			embedding = torch.as_tensor(embedding, dtype=torch.float)#.cuda(self.device)


			#--------------------------------------------------
			# print(self.list_of_data[idx])
			mask = np.load(self.ground_truth_path + self.list_of_data[idx].split("/")[7] + "/" +  self.list_of_data[idx].split("/")[9])
		#   mask[mask>0] = 1
			size = mask.shape
			box = get_bounding_box(mask)
			mask = mask.astype(np.float32)
			# mask = squarify(mask)





			transform_r = ResizeLongestSide(256)
			mask = transform_r.apply_image(mask)
			mask = torch.as_tensor(mask)
			mask = preprocess(mask)

			class_1 = np.zeros_like(mask)
			class_2 = np.zeros_like(mask)
			class_3 = np.zeros_like(mask)
			# 1 is core 2 is edema 3 is enhancing
			class_1[np.where(mask == 1)] = 1
			class_1[np.where(mask == 2)] = 1
			class_1[np.where(mask == 3)] = 1

			class_2[np.where(mask == 1)] = 1
			class_2[np.where(mask == 3)] = 1

			class_3[np.where(mask == 1)] =  1

			mask = np.stack((class_1, class_2, class_3), axis = 0 )



			mask = torch.as_tensor(mask)#.cuda(self.device)
		#   mask = mask[None,...]


			# box = get_bounding_box(mask)
			# box = [0,0,mask.shape[-2],mask.shape[-1]]
			box_np = np.array(box)
			sam_trans = ResizeLongestSide(self.sam_model.image_encoder.img_size)
			box = sam_trans.apply_boxes(box_np, (size[0], size[1]))
			box_torch = torch.as_tensor(box, dtype=torch.float)#.cuda(self.device)
			# box_torch = box_torch[None, :]

			with torch.no_grad():
			# embedding = sam_model.image_encoder(image)
				sparse_embeddings, dense_embeddings = self.sam_model.prompt_encoder(
					points=None,
					boxes=box_torch,
					masks=None,
			)


			sample = {'embedding': embedding,
								'mask': mask,
								# 'image_pe' : sam_model.prompt_encoder.get_dense_pe(),
								'sparse_embeddings': torch.squeeze(sparse_embeddings.detach(), axis = 0),
								'dense_embeddings': torch.squeeze(dense_embeddings.detach(), axis = 0),
									}

			return sample


# train = BraTSDataset(ground_truth_path, embedding_path, train_list,sam_model= "n")
# train[0]

In [ ]:
torch.cuda.empty_cache()
def eval(val_dataloader, sam_model, seg_loss, device, image_pe):
	loss = 0
	for step_val, data in enumerate(tqdm(val_dataloader)):
		embedding, mask, sparse_embeddings, dense_embeddings = data["embedding"], data["mask"], data["sparse_embeddings"], data["dense_embeddings"]


		with torch.no_grad():
			# predicted masks
			mask_predictions, _ = sam_model(
			image_embeddings= embedding.to(device), # (B, 256, 64, 64)
			image_pe=image_pe.to(device), # (1, 256, 64, 64)
			sparse_prompt_embeddings=sparse_embeddings.to(device), # (B, 2, 256)
			dense_prompt_embeddings=dense_embeddings.to(device), # (B, 256, 64, 64)
			multimask_output=True,
			)

			loss += seg_loss(mask_predictions, mask.to(device))
	loss /= step_val+1
	return loss

def train():
  hprams = {
			"num_epochs": 90,
			"best_loss": 1e10,
			"model_save_path": "/content/drive/MyDrive/save_to/sam",
			"lr":1e-5,
			"weight_decay":0,

		}




  model_type = 'vit_h'
  checkpoint = 'sam_vit_h_4b8939.pth'
  device = "cuda"
  sam_model = sam_model_registry[model_type](checkpoint=checkpoint)#.to(device)
  with torch.no_grad():
    get_dense_pe = sam_model.prompt_encoder.get_dense_pe()

  train = BraTSDataset(ground_truth_path, embedding_path, train_list,sam_model)
  val = BraTSDataset(ground_truth_path, embedding_path, val_list, sam_model)


  train_loader = DataLoader(train, batch_size= 64, shuffle=True, num_workers=12, prefetch_factor=30)
  val_loader = DataLoader(val, batch_size= 64, shuffle=False, num_workers=12, prefetch_factor=30)
  print("data_loaded")

  model_save_path = hprams['model_save_path']
  num_epochs = hprams['num_epochs']
  best_loss = hprams['best_loss']
  lr = hprams['lr']
  weight_decay = hprams['weight_decay']

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  os.makedirs(model_save_path, exist_ok=True)

  model = sam_model.mask_decoder
  model.to(device)


  model.train()

  optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, factor=0.25,  patience=3)
  seg_loss = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, reduction='mean').to(device)

  # wandb.init(project = "SAM Finetune",
  #         save_code = True, group= 'Decoder_multiclass_1')

  for epoch in range(num_epochs):
      epoch_loss = 0
      # train
      for step, data in enumerate(tqdm(train_loader)):
          embedding, mask, sparse_embeddings, dense_embeddings = data["embedding"], data["mask"], data["sparse_embeddings"], data["dense_embeddings"]

          mask_predictions, _ = model(
            image_embeddings= embedding.to(device), # (B, 256, 64, 64)
            image_pe=get_dense_pe.to(device), # (1, 256, 64, 64)
            sparse_prompt_embeddings=sparse_embeddings.to(device), # (B, 2, 256)
            dense_prompt_embeddings=dense_embeddings.to(device), # (B, 256, 64, 64)
            multimask_output=True,
          )

          loss = seg_loss(mask_predictions, mask.to(device))
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          epoch_loss += loss.item()
          # break
      epoch_loss /= step+1
      # wandb.log({"loss": epoch_loss})

      val_loss = None
      if val_loader is not None: val_loss = eval(val_loader, model, seg_loss, device, get_dense_pe)
      print(f'EPOCH: {epoch}, Loss: {epoch_loss}, Val loss {val_loss}')
      scheduler.step(val_loss)
      # wandb.log({"val_loss": val_loss})
      # wandb.log({"epoch": epoch, "loss": loss})

      # save the latest model checkpoint
      # torch.save(sam_model.state_dict(), os.path.join(model_save_path, 'sam_model_'+str(epoch)+'.pth'))

      # save the best model

      if val_loss < best_loss:
          best_loss = val_loss
          torch.save(model.state_dict(), os.path.join(model_save_path, 'sam_model_best_multi2.pth'))

      # if epoch>=3 and epoch<=5:
      #   for g in optimizer.param_groups:
      #     g['lr'] = g['lr']*10
      #   print(f"worm up lr is {g['lr']}")

      torch.save(model.state_dict(), os.path.join(model_save_path, 'sam_model_last_multi2.pth'))





train()

data_loaded


100%|██████████| 46/46 [03:01<00:00,  3.95s/it]


EPOCH: 0, Loss: 0.6484832966102744, Val loss 0.5281452536582947


100%|██████████| 46/46 [04:20<00:00,  5.67s/it]


EPOCH: 1, Loss: 0.5902375663226506, Val loss 0.5263969898223877


...

EPOCH: 19, Loss: 0.33793152756286116, Val loss 0.4173314571380615


100%|██████████| 46/46 [02:40<00:00,  3.49s/it]


EPOCH: 20, Loss: 0.33790318949042625, Val loss 0.41229015588760376


 11%|█▏        | 12/106 [1:57:53<15:23:29, 589.47s/it]


KeyboardInterrupt: ignored

In [ ]:
# ! cp -r "/content/drive/MyDrive/miccai_data/BraTS_data_africa" "/content"

Streaming output truncated to the last 5000 lines.
cp: cannot stat '/content/drive/MyDrive/miccai_data/BraTS_data_africa/t2/z/embeddings/BraTS-SSA-00112-000_16.npy': No such file or directory
cp: cannot stat '/content/drive/MyDrive/miccai_data/BraTS_data_africa/t2/z/embeddings/BraTS-SSA-00112-000_17.npy': No such file or directory
cp: cannot stat '/content/drive/MyDrive/miccai_data/BraTS_data_africa/t2/z/embeddings/BraTS-SSA-00112-000_18.npy': No such file or directory
cp: cannot stat '/content/drive/MyDrive/miccai_data/BraTS_data_africa/t2/z/embeddings/BraTS-SSA-00112-000_19.npy': No such file or directory
cp: cannot stat '/content/drive/MyDrive/miccai_data/BraTS_data_africa/t2/z/embeddings/BraTS-SSA-00112-000_1.npy': No such file or directory
cp: cannot stat '/content/drive/MyDrive/miccai_data/BraTS_data_africa/t2/z/embeddings/BraTS-SSA-00112-000_20.npy': No such file or directory
cp: cannot stat '/content/drive/MyDrive/miccai_data/BraTS_data_africa/t2/z/embeddings/BraTS-SSA-00112-00

In [ ]:
rm -r "/content/BraTS_data_africa"

In [ ]:
import os
len(os.listdir("/content/BraTS_data_africa/t2/z/embeddings"))

68

In [ ]:
import pandas as pd
pd.read_json("res (1).json").transpose().to_excel("output.xlsx")